# Set puzzle parameters and create AoC Session

In [ ]:
# set puzzle parameters
PUZZLE_DAY = 9
PUZZLE_YEAR = 2024

# import from local packages
from aoc_solver import AoCSession, AoCSolver, AoCTester
AoC_SESSION = AoCSession.from_file()

# Import additional packages

In [ ]:
# import from standard library packages
from typing import Tuple

# import from third-party packages
import polars
from polars import Series
from tqdm.notebook import tqdm

# Create solver class and instance

In [ ]:
class Solver(AoCSolver):
    def solve_part1(self, data: Tuple[str]) -> int:
        filesystem_length = sum(int(i) for i in data)
        filesystem = Series(['.']*filesystem_length)
        
        file_id = 0
        current_index = 0
        open_blocks = []
        for i, record in enumerate(tqdm(data)):
            current_blocks = list(range(current_index, current_index+record))
            if i % 2 == 0:
                filesystem[current_blocks] = Series([file_id]*record)
                file_id += 1
            else:
                open_blocks.extend(current_blocks)
            current_index += record
        
        open_blocks_index = 0
        for i in tqdm(range(filesystem_length-1, -1, -1)):
            if filesystem[i] == '.':
                continue
            if i <= open_blocks[open_blocks_index]:
                break
            filesystem[open_blocks[open_blocks_index]] = filesystem[i]
            filesystem[i] = '.'
            open_blocks_index += 1
        return sum(i*int(x) for i,x in enumerate(filesystem) if x != '.')

    def solve_part2(self, data: Tuple[str]) -> int:
        filesystem_length = sum(int(i) for i in data)
        filesystem = Series(['.']*filesystem_length)
        
        file_id = 0
        current_index = 0
        open_blocks = {}  # {starting block: length}
        file_locations = {}  # {file_name: (starting block, length)}
        for i, record in enumerate(tqdm(data)):
            current_blocks = list(range(current_index, current_index+record))
            if i % 2 == 0:
                filesystem[current_blocks] = Series([file_id]*record)
                file_locations[file_id] = (current_index, record)
                file_id += 1
            elif record > 0:
                open_blocks[current_index] = record
            current_index += record
        
        file_locations = dict(sorted(file_locations.items(), reverse=True))
        for file_id, (file_location, file_length) in tqdm(file_locations.items()):
            try:
                open_block_location = min(
                    k for k in open_blocks.keys()
                    if open_blocks[k] >= file_length and k < file_location
                )
            except ValueError:
                continue
            
            # copy file to new location
            new_file_indices = list(range(
                open_block_location,
                open_block_location+file_length
            ))
            filesystem[new_file_indices] = Series([file_id]*file_length)
            open_block_length = open_blocks.pop(open_block_location)
            open_block_indices = list(range(
                open_block_location,
                open_block_location+open_block_length
            ))
            if len(open_block_indices) > file_length:
                open_block_location += file_length
                open_block_length -= file_length
                open_blocks[open_block_location] = open_block_length
            
            # erase blocks for previous file location
            new_open_indices = list(range(file_location, file_location+file_length))
            filesystem[new_open_indices] = Series(['.']*file_length)
            if (i := new_open_indices[-1]+1) in open_blocks.keys():
                new_open_indices.extend(range(i, i+open_blocks.pop(i)))
            try:
                i = max(
                    k for k in open_blocks.keys() if k < new_open_indices[0]
                )
                lh_adjustment = True
            except ValueError:
                lh_adjustment = False
            if lh_adjustment and (i + open_blocks[i] == new_open_indices[0]):
                new_open_indices.extend(range(i, i+open_blocks.pop(i)))
            open_blocks[sorted(new_open_indices)[0]] = len(new_open_indices)
            
        return sum(i*int(x) for i,x in enumerate(filesystem) if x != '.')

In [ ]:
solver = Solver(PUZZLE_YEAR, PUZZLE_DAY, AoC_SESSION)

# Build part 1 test case(s)

In [ ]:
puzzle_instructions = solver.puzzle_instructions
puzzle_input = solver.puzzle_input.create_tuple(dtype=int ,separator='')

part1_test_input = solver.get_value_after('For example:').create_tuple(dtype=int ,separator='')
print(f'{part1_test_input=}\n')

part1_test_output = solver.get_value_after('is the sum of these, ').as_int
print(f'{part1_test_output=}\n')

In [ ]:
part_1_tester = AoCTester()
part_1_tester.add_test_case(part1_test_input, part1_test_output)

In [ ]:
%%time
part_1_tester.run_tests(solver.solve_part1)

# Determine part 1 solution

In [ ]:
%%time
part1_solution = solver.solve_part1(puzzle_input)
print(f'{part1_solution=}\n')

# Add part 1 solution to part 1 test cases

In [ ]:
part_1_tester.add_test_case(puzzle_input, part1_solution)

In [ ]:
%%time
part_1_tester.run_tests(solver.solve_part1)

# Build part 2 test case(s)

In [ ]:
solver.download_instructions(overwrite=True)

In [ ]:
part2_test_input = part1_test_input
print(f'{part2_test_input=}\n')

part2_test_output = solver.get_value_after('example\'s checksum would be ').as_int
print(f'{part2_test_output=}\n')

In [ ]:
part_2_tester = AoCTester()
part_2_tester.add_test_case(part2_test_input, part2_test_output)

In [ ]:
%%time
part_2_tester.run_tests(solver.solve_part2)

# Determine part 2 solution

In [ ]:
%%time
part2_solution = solver.solve_part2(puzzle_input)
print(f'{part2_solution=}\n')